In [1]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin


## Initial data

In [2]:
db_pomoci = pd.read_csv("db_pomoci.csv")
black_list = ['http://www.dc-brno.cz']
df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev"})
df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
# df['web'] = df['web'].str.replace('https://www', 'www')
# df['web'] = df['web'].str.replace('www', 'https://www')
df_agg = df.groupby("web")["Telefon"].agg(list)
df_agg["email"] = df.groupby("web")["E-mail"].agg(list)
df_agg = df_agg.reset_index().rename(columns={'web': 'web', 'Telefon': 'Telefon'})
df_agg = pd.DataFrame(df_agg)

# Scraped data

In [3]:
db_scraped = pd.read_csv("data/df_scraped.csv")
df_res = db_scraped[~db_scraped["Phone Numbers"].isna()]
df_res = df_res.rename(columns={'web': 'web', 'Telefon': 'Telefon'}).drop("Unnamed: 0", axis= 1)
df_res = df_res.rename(columns= {"Website":"web"})
df_res = df_res.rename(columns= {"Phone Numbers":"phones_scraped"})

# Mapy.cz

In [13]:
maps_df = pd.read_csv("data/maps_results.csv",sep=";")
maps_df = maps_df.rename(columns={'Web': 'web', 'Phone': 'Telefon Mapy'})
mapy_tel = maps_df[["web","Telefon Mapy"]]

In [14]:
preview = df_res.merge(df_agg , on="web")
preview = preview.merge(mapy_tel , on="web")
preview['phones_scraped'] = preview['phones_scraped'].str.split(', ')
preview


,web,phones_scraped,Telefon,Telefon Mapy
0,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]",778 526 216
1,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]","469 626 383, 774 915 755"
2,http://www.medsix.cz/?option=com_content&view=...,"[483 341 232, 483 300 087, 483 310 919, 483 34...",[483 341 267],483 341 267
3,https://jindrichuvhradec.charita.cz/sluzba/852/,"[731 402 982, 731 604 554, 605 849 877]","[731 402 982, 605 849 877]","731 402 982, 605 849 877"
4,https://www.homeopatiecz.com/,"[777 169 278, 736 660 315, 777 146 760, 725 76...",[736 660 315],736 660 315
...,...,...,...,...
612,https://www.firmy.cz/detail/13179143-phdr-tere...,[420 376 322 688],[376 322 688],376 322 688
613,https://search.seznam.cz/?q=psychiatrick%C3%A1...,[420608971030],[736 132 143],736 132 143
614,http://ssp-ol.cz/,[585 427 141],[774 406 453],774 406 453
615,http://www.psychoterapeuti.cz/adresar-psychote...,"[284 016 451, 736 660 315, 530 334 600, 775 18...",[466 611 601],466 611 601
